## Dealing with images

We'll take a look at the Python Imaging Library. The main points about this library:

- Reads and writes essentially any image format.
- Can do all sorts of manipulation (resizing, cropping, enhancing, drawing, compositing).
- Can convert to and from NumPy arrays.

We'll also briefly look at reading and writing images with `matplotlib` and `scikit-image`. Main features of `matplotlib`:

- Only reads PNG format.
- Doesn't do any manipulation, although you can resize an image in `scipy`.
- Writes most common formats (PNG, JPG, GIF, plus PDF and SVG for vectors).
- Uses NumPy arrays implicitly.

To install it, do:

    conda install pillow

Start with the usual:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## A quick look at `matplotlib`

[Here are the docs.](https://matplotlib.org/users/image_tutorial.html)

Matplotlib can only load PNGs natively. You can load a JPEG, but mpl will use PIL behind the scenes.

Unlike PIL, mpl loads images as 4-byte floats in [0, 1]. 

First, download this image to your computer and then use the filename to load it with `matplotlib`:

<img src="https://geocomp.s3.amazonaws.com/data/quartz-arenite_Michael-C-Rygel_CC-BY-SA.png" width=600 />

In [ ]:
img = plt.imread("PATH/TO/THE/FILE")

In [ ]:
import numpy as np

np.asarray(img)

Notice we have an h &times; w &times; 3 array — this is an RGB image. PNGs often have a 4th channel, alpha or A, which holds opacity.

`plt.imshow()` plots 3-channel arrays like this in colour:

In [ ]:
plt.imshow(img)

We can plot only the red channel (say), and apply false colour via a lookup table:

In [ ]:
plt.imshow(img[..., 0])
plt.colorbar()

In [ ]:
_ = plt.hist(img[..., 0].ravel(), bins=128)

## PIL's `Image` object

[Here are the docs!](http://effbot.org/imagingbook/image.htm)

In [ ]:
from PIL import Image
import urllib

url = "https://geocomp.s3.amazonaws.com/data/quartz-arenite_Michael-C-Rygel_CC-BY-SA.png"
im = Image.open(urllib.request.urlopen(url))

In [ ]:
im

Notice that the size (not `shape`!) is reported as columns &times; rows, so it's different from a NumPy array.

In [ ]:
im.size

In [ ]:
np.array(im).shape

In [ ]:
aspect = 1200/961
w = 600
h = int(w / aspect)   # Has to be an int.

im = im.resize((w, h), Image.ANTIALIAS)
im

You can save having to compute the new image size with the `thumbnail` method but be careful — it resizes the image in place:

In [ ]:
temp = im.copy()
temp.thumbnail((64, 64), Image.ANTIALIAS)

In [ ]:
temp

We can plot this little image and see that it's now pixellated at any reasonable size:

In [ ]:
plt.imshow(temp, interpolation='none')

We can ask `imshow` for some more sensible interpolation:

In [ ]:
plt.imshow(temp, interpolation='bicubic')

## Handling images as arrays

As you see in the last example, we can treat PIL `Image` as an array sometimes, eg passing it to `imshow`. But sometimes it's convenient to treat images entirely as NumPy arrays. It's easy to convert between the two:

In [ ]:
rgb = np.array(im)

red_channel = rgb[:, :, 0]
plt.imshow(red_channel, cmap='gray')

Note that NumPy doesn't implicitly care about the values:

In [ ]:
np.max(red_channel)

In [ ]:
red_max1 = red_channel / 255
plt.imshow(red_max1, cmap='gray')

But if you convert back to a PIL `Image`, it cares. In fact, it won't even accept our decimal numbers in the 0–1 range:

In [ ]:
im_red =  Image.fromarray(red_max1)
im_red

We have to cast them to unsigned 8-bit integers (i.e. integers in the range 0 to 255):

In [ ]:
im_red = Image.fromarray(np.uint8(red_max1 * 255))
im_red

### Exercise

- Make a histogram of the image using [the `histogram()` method](http://effbot.org/imagingbook/image.htm#tag-Image.Image.histogram) on `Image`.
- Use the `crop()` method on the `Image` to crop the scalebar off the image.
- Crop the scalebar out of the image, and calculate the real-world size of each pixel in the image.
- Compute the luminance of the image $Y = 0.299 R + 0.587 G + 0.114 B$ and plot a transect across the middle of the image.

In [ ]:
# Your code here...

### Histogram

In [ ]:
im.histogram?

In [ ]:
h = np.array(im.histogram())
r, g, b = np.array_split(h, 3)

plt.plot(r, 'r')
plt.plot(g, 'g')
plt.plot(b, 'b')
plt.show()

In [ ]:
im.size

### Cropping

In [ ]:
# left, upper, right, lower
main = im.crop((2, 2, 530, 435))
main

In [ ]:
scale = im.crop((530, 435, 600, 480))
scale = np.array(scale)

plt.imshow(scale)

In [ ]:
plt.plot(scale[10, :, 0])

In [ ]:
bar = scale[10, :, 0] < 100
bar.sum()

Therefore we have 57 pixels in the scale bar. We know it's 0.25 mm or 0.25 &times; 10<sup>–3</sup> m long, so each pixel is:

In [ ]:
0.25e-3 / 57

... about 4.4 microns wide.

### Luminance

In [ ]:
r, g, b = np.array(main).T
y = 0.299*r + 0.587*g + 0.114*b

In [ ]:
plt.imshow(y.T, cmap='gray')

Save this file as `"../data/quartz-arenite_LUMINANCE_Michael-C-Rygel_CC-BY-SA.npy"`

In [ ]:
fname = "../data/quartz-arenite_LUMINANCE_Michael-C-Rygel_CC-BY-SA.npy"
np.save(fname, y.T)

## Where next?

Next, try the tutorial on [Image segmentation](../demos/Image_segmentation.ipynb).